In [4]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import re
from transformers import AutoTokenizer

In [7]:
def analyze_dialogue_lengths(df):
    """대화 길이 분포 분석"""
    tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
    
    # 토큰 길이 계산
    token_lengths = []
    for text in df['dialogue']:
        tokens = tokenizer.encode(text)
        token_lengths.append(len(tokens))
    
    # 길이 통계
    length_stats = {
        'mean': np.mean(token_lengths),
        'median': np.median(token_lengths),
        'p95': np.percentile(token_lengths, 95),
        'max': np.max(token_lengths),
        'min': np.min(token_lengths)
    }
    
    return length_stats, token_lengths

def analyze_dialogue_patterns(df):
    """대화 패턴 분석"""
    # 감정 표현 패턴
    emotion_patterns = {
        'happy': r'(좋아|기쁘|행복|즐거|웃)',
        'sad': r'(슬프|우울|안타깝|힘들)',
        'angry': r'(화나|짜증|싫|못마땅|불편)',
        'neutral': r'(그렇군요|알겠습니다|네)'
    }
    
    # 대화 의도 패턴
    intent_patterns = {
        'question': r'(\?|궁금|어떻|무엇|언제|어디|누구)',
        'suggestion': r'(제안|추천|하는 게 어떨까|면 좋겠|면 어떨까)',
        'opinion': r'(생각|의견|보면|같아)'
    }
    
    # 패턴 빈도수 계산
    pattern_counts = {
        'emotions': defaultdict(int),
        'intents': defaultdict(int)
    }
    
    for text in df['dialogue']:
        # 감정 패턴 확인
        for emotion, pattern in emotion_patterns.items():
            if re.search(pattern, text):
                pattern_counts['emotions'][emotion] += 1
        
        # 의도 패턴 확인
        for intent, pattern in intent_patterns.items():
            if re.search(pattern, text):
                pattern_counts['intents'][intent] += 1
    
    return pattern_counts, emotion_patterns, intent_patterns

def insert_special_tokens(text, emotion_patterns, intent_patterns):
    """패턴 분석 결과를 바탕으로 special tokens 삽입"""
    modified_text = text
    
    # 감정 패턴에 따른 토큰 삽입
    for emotion, pattern in emotion_patterns.items():
        if re.search(pattern, text):
            modified_text = f"#{emotion.capitalize()}# " + modified_text
    
    # 의도 패턴에 따른 토큰 삽입
    for intent, pattern in intent_patterns.items():
        if re.search(pattern, text):
            modified_text = f"#{intent.capitalize()}# " + modified_text
    
    # 화자 전환 표시
    turns = modified_text.split('\n')
    processed_turns = []
    for turn in turns:
        if turn.strip():
            processed_turns.append(f"#Turn# {turn}")
    
    return '\n'.join(processed_turns)

def process_dataset(df):
    """전체 데이터셋 처리"""
    # 길이 분석
    length_stats, token_lengths = analyze_dialogue_lengths(df)
    print("Dialogue Length Statistics:")
    print(length_stats)
    
    # 패턴 분석
    pattern_counts, emotion_patterns, intent_patterns = analyze_dialogue_patterns(df)
    print("\nPattern Frequencies:")
    print(pattern_counts)
    
    # 가장 빈번한 패턴들을 기반으로 special tokens 결정
    recommended_tokens = []
    for category, counts in pattern_counts.items():
        # 일정 빈도 이상 나타나는 패턴만 선택
        threshold = len(df) * 0.05  # 5% 이상 출현하는 패턴만 선택
        frequent_patterns = [k for k, v in counts.items() if v > threshold]
        recommended_tokens.extend([f"#{p.capitalize()}#" for p in frequent_patterns])
    
    print("\nRecommended Special Tokens:")
    print(recommended_tokens)
    
    # optimal encoder_max_len 추천
    recommended_max_len = int(np.percentile(token_lengths, 95))  # 95퍼센타일 값 사용
    print(f"\nRecommended encoder_max_len: {recommended_max_len}")
    
    # 토큰 삽입 예시
    sample_processed = insert_special_tokens(df['dialogue'].iloc[0], emotion_patterns, intent_patterns)
    print("\nProcessed Text Example:")
    print(sample_processed)
    
    return recommended_tokens, recommended_max_len

In [9]:
# 사용 예시:
df = pd.read_csv('/root/NLP/data/sum_aug_bt.csv')
recommended_tokens, recommended_max_len = process_dataset(df)

Dialogue Length Statistics:
{'mean': 206.41262069903442, 'median': 188.0, 'p95': 378.0, 'max': 1324, 'min': 29}

Pattern Frequencies:
{'emotions': defaultdict(<class 'int'>, {'neutral': 20476, 'sad': 1321, 'happy': 10534, 'angry': 1325}), 'intents': defaultdict(<class 'int'>, {'question': 28338, 'opinion': 13160, 'suggestion': 2811})}

Recommended Special Tokens:
['#Neutral#', '#Happy#', '#Question#', '#Opinion#', '#Suggestion#']

Recommended encoder_max_len: 378

Processed Text Example:
#Turn# #Opinion# #Question# #Neutral# #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Turn# #Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Turn# #Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Turn# #Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Turn# #Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Turn# #Person2#: 알겠습니다.
#Turn# #Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Turn# #Person2#: 네.
#Turn# #Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인